In [1]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [3]:
%%time
infile = open('./data/500_MC_dgp2_uts.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]']
print(Y_dgp.shape)

(2798, 500)
Wall time: 31.9 ms


In [ ]:
%%time
result = []
draws = 250
start = 0
end = 50
method = 'SMC'
for i in range(start,end):
    print('iteration: %i' %(i+1))
    qmc = MMNL(X,Y_dgp[:,i],draws,3,method=method)
    solution = qmc.solver()
    result.append(solution)
    if i%10==9:
        pickle.dump( result, open( "./resultaten/%s(%i)/%i-%i_%s(%i)_utsgdp_results.p" %(method,draws,i-9,i,method,draws), "wb" ) )
        result = []
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

iteration: 1


In [11]:
pickle.dump( result, open( "./resultaten/BQMC(8)/90_BQMC(8)_results.p", "wb" ) )

In [10]:
x_est = np.array(result)
x_est.shape

(100, 6)

In [11]:
np.mean(x_est, axis=0)

array([ 4.58777441,  2.65487859, -7.12572568,  7.45172249,  2.22983804,
        5.23629935])